#### Ingestion del archivo movie_company

In [0]:
dbutils.widgets.text("p_enviroment","")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

##### Paso 1 - Leer el archivo Json usando "DataFrameReader" en Spark

In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType

In [0]:
movie_company_schema = StructType(fields = [
  StructField("movieId", IntegerType(), False),
  StructField("companyId", IntegerType(),False)
])

In [0]:
movie_company_df = spark.read.schema(movie_company_schema).csv(f"{bronze_folder_path}/{v_file_date}/movie_company")

#### Paso 2 - Agregar los campos faltnates y cambiar de nombre
##### 1. Cambiar el nombre de "movieId" a "movie_id"
##### 2. Cambiar el nombre de "companyId" a "company_id"
##### 2. Agregar "enviroment" y Agregar "ingestion_date"

In [0]:
from pyspark.sql.functions import lit, current_timestamp

In [0]:
movie_company_final = add_ingestion_date(movie_company_df).withColumnRenamed("movieId","movie_id") \
                                .withColumnRenamed("companyId","company_id") \
                                .withColumn("enviromen",lit(v_enviroment)) \
                                .withColumn("file_date",lit(v_file_date))

#### Paso 3 - Agregar el DF en formato parQuet

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.company_id = src.company_id AND tgt.file_date = src.file_date'
merge_delta_lake(movie_company_final, "movie_silver", "movie_company", silver_foler_path, merge_condition, "file_date")

#overwrite_partition(movie_company_final,"movie_silver","mive_company","file_date")

In [0]:
%sql
SELECT COUNT(1), file_date
FROM movie_silver.movie_company
GROUP BY file_date

In [0]:
dbutils.notebook.exit("Success!")